In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import time
import json
import urllib.parse
import re
import pickle
import os

cookies_path = "cookies.pkl"

difficulty_map = {'入门':'easy','普及−':'easy', 
                  '普及+/提高':'medium','普及/提高−': 'medium', 
                  '提高+/省选−':'medium',
                  '省选/NOI−':'hard', 'NOI/NOI+/CTSC':'hard',
                  '暂无评定':'unknown'}
# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URLs
login_url = "https://www.luogu.com.cn/auth/login"
base_problem_url = "https://www.luogu.com.cn/problem/list?type=SP&page="
problem_detail_url = "https://www.luogu.com.cn/problem/"
solution_url = "https://www.luogu.com.cn/problem/solution/"

# Your login credentials
username = 'kathcym'
password = 'Kath_CYM.017'

# Function to save cookies after logging in
def save_cookies(driver, filepath):
    with open(filepath, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

# Function to load cookies and reuse the session
def load_cookies(driver, filepath):
    with open(filepath, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            driver.add_cookie(cookie)

# Login to the site
if os.path.exists(cookies_path):
    driver.get(login_url)
    load_cookies(driver, cookies_path)
    driver.refresh()
else:
    driver.get(login_url)
    time.sleep(30)  # Wait for manual login to complete
    save_cookies(driver, cookies_path)
    print('login from cookies')
    driver.refresh()

exit()
# Continue with the rest of the script as before
driver.get(base_problem_url + "1")
time.sleep(2)  # Wait for the page to load

# Extract total number of pages dynamically
total_pages_element = driver.find_element(By.CLASS_NAME, "total").find_element(By.TAG_NAME, "strong")
total_pages = int(total_pages_element.text.strip())
print('total_pages', total_pages)

resume = False
def remove_code_blocks(text):
    # This regex finds code blocks enclosed within triple backticks
    return re.sub(r'```[a-zA-Z]*\n[\s\S]*?\n```', '', text)

def extract_code_blocks(text):
    # This regex finds code blocks enclosed within triple backticks
    code_blocks = re.findall(r'```[a-zA-Z]*\n([\s\S]*?)\n```', text)
    return code_blocks

# Iterate through pages
for page_num in range(1, total_pages + 1):
    problems_list = []
    driver.get(base_problem_url + str(page_num))
    time.sleep(2)
    print('page:', page_num)
    problem_rows = driver.find_elements(By.CLASS_NAME, 'row')
    problems_info = []
    
    for row in problem_rows:
        problem_id = row.text.split()[0]
        #if problem_id == 'CF1859D':
        #    resume = True
        title = ' '.join(row.text.split('\n')[0].split()[1:])
        difficulty = difficulty_map[row.text.split('\n')[1]]
        #if resume:
        problems_info.append((problem_id, title, difficulty))

    for problem_id, title, difficulty in problems_info:
        driver.get(problem_detail_url + problem_id)
        time.sleep(2)

        page_source = driver.page_source
        
        search_string = '<script>window._feInjection = JSON.parse(decodeURIComponent('
        found_line = None
        for line in page_source.splitlines():
            if search_string in line:
                found_line = line
                break
        if found_line:
            blocks = found_line.split('));')[0].replace(search_string, '').replace('"', '').strip()
            blocks = json.loads(urllib.parse.unquote(blocks))
        problem_content = (blocks['currentData']['problem']['background'] or '') + \
                          (blocks['currentData']['problem']['description'] or '') + \
                          (blocks['currentData']['problem']['inputFormat'] or '') + \
                          (blocks['currentData']['problem']['outputFormat'] or '')
        #problem_content = driver.find_element(By.CLASS_NAME, 'problem-card').text.strip()
        #problem_content = problem_content.replace('复制Markdown 展开', '').replace('复制\n', '')
        #input_output_pairs = re.findall(r'输入\s+#\d+\n(.*?)\n输出\s+#\d+\n(.*?)(?=\n说明/提示|\n输入\s+#\d+|\Z)', problem_content, re.DOTALL)
        difficulty_rating = blocks['currentData']['problem']['difficulty']
        memory_limit = blocks['currentData']['problem']['limits']['memory']
        time_limit = blocks['currentData']['problem']['limits']['time']
        tags = blocks['currentData']['problem']['tags']
        public_test_cases = []
        
        for c in blocks['currentData']['problem']['samples']:
            test_case = {
                "input": c[0],
                "output": c[1],
                "type": "stdin"
            }
            public_test_cases.append(test_case)
        #print(problem_content)
        #print(public_test_cases)
        
        # Fetch solution content
        driver.get(solution_url + problem_id + '?orderBy=weight')
        time.sleep(2)
        page_source = driver.page_source
        
        search_string = '<script>window._feInjection = JSON.parse(decodeURIComponent('
        found_line = None
        for line in page_source.splitlines():
            if search_string in line:
                found_line = line
                #print(found_line)
        if found_line:
            blocks = found_line.split('));')[0].replace(search_string, '').replace('"', '').strip()
            #print(blocks)
            blocks = json.loads(urllib.parse.unquote(blocks))
            #print(blocks)
            solutions = blocks['currentData']['solutions']['result']
        else:
            print("Solutions blocks not found.")
        if len(solutions) == 0:
            continue
        solution_content = remove_code_blocks(solutions[0]['content'])
        solution_code = extract_code_blocks(solutions[0]['content'])
        #vote = solution['thumbUp']
        #solutions_list.append((vote, content, code))

        #solutions_sorted = sorted(solutions_list)
        #if len(solutions_sorted) > 5:
        #    solutions_list = solutions_sorted[:5]
        #print(solution_content)
        #print(solution_code)
        problem = {
            'question_id': problem_id,
            'question_title': title,
            'platform': 'SPOJ',
            'question_content': problem_content,
            'difficulty': difficulty,
            'public_test_cases': public_test_cases,
            'contest_id': 'SPOJ',
            'starter_code': '',
            'contest_date': '',
            'meta_data': {'difficulty_rating': difficulty_rating,
                          'memory_limit': memory_limit,
                          'time_limit': time_limit,
                          'tags': tags,
                          'nl_solution': solution_content, 
                          'code_solution':solution_code}
        }
        print(problem_id, '-', title)
        problems_list.append(problem)
    jsonl_file_path = 'luogu-spoj/data.jsonl'

    # Write the JSON objects to the .jsonl file
    with open(jsonl_file_path, 'a+') as file:
        for p in problems_list:
            file.write(json.dumps(p) + '\n')
    
    print(f"{len(problems_list)} samples saved to {jsonl_file_path}")
    #with open(jsonl_file_path, 'r') as file:
    #    json_objects = [json.loads(line) for line in file]
    #print('last example', json_objects[-1])
driver.quit()

total_pages 74
page: 1
SP1 - TEST - Life, the Universe, and Everything
SP2 - PRIME1 - Prime Generator
SP4 - ONP - Transform the Expression
SP5 - PALIN - The Next Palindrome
SP6 - ARITH - Simple Arithmetics
SP7 - BULK - The Bulk!
SP8 - CMPLS - Complete the Sequence!
SP9 - DIRVS - Direct Visibility
SP10 - CMEXPR - Complicated Expressions
SP11 - FCTRL - Factorial
SP13 - HOTLINE - Hotline
SP15 - SHPATH - The Shortest Path
SP16 - TETRA - Sphere in a tetrahedron
SP17 - CRYPTO1 - The Bytelandian Cryptographer (Act I)
SP19 - CRYPTO3 - The Bytelandian Cryptographer (Act III)
SP22 - TRICENTR - Triangle From Centroid
SP23 - PIR - Pyramids
SP24 - FCTRL2 - Small factorials
SP25 - POUR1 - Pouring water
SP27 - SBANK - Sorting Bank Accounts
SP28 - HMRO - Help the Military Recruitment Office!
SP29 - HASHIT - Hash it!
SP30 - BLINNET - Bytelandian Blingors Network
SP31 - MUL - Fast Multiplication
SP32 - NHAY - A Needle in the Haystack
SP33 - TRIP - Trip
SP34 - RUNAWAY - Run Away
SP35 - EQBOX - Equipment 

KeyboardInterrupt: 